<a href="https://colab.research.google.com/github/Bouchendira/CombForest/blob/main/Balacing_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Original for testing perposes

In [ ]:

import random
from collections import defaultdict

def generate_random_graph(num_nodes, num_edges):
    graph = {chr(65 + i): [] for i in range(num_nodes)}
    if num_edges > num_nodes * (num_nodes - 1) // 2:
        print("Warning: Number of edges exceeds maximum possible edges for this number of nodes.")
        num_edges = num_nodes * (num_nodes - 1) // 2

    edges = set()
    keys = list(graph.keys())
    while len(edges) < num_edges:
        u = random.choice(keys)
        v = random.choice(keys)
        if u != v and (u, v) not in edges and (v, u) not in edges:
            edges.add((u, v))
            graph[u].append(v)
            graph[v].append(u)

    return graph
def find_left_end(curr, CombForest):
    # ascend left to the leftmost node (stop when parentleft is None)
    while CombForest[curr]['parentleft'] is not None:
        curr = CombForest[curr]['parentleft']
    return curr

def find_right_end(curr, CombForest):
    while CombForest[curr]['parentright'] is not None:
        curr = CombForest[curr]['parentright']
    return curr

# --- Example random graph ---
num_nodes = 10
num_edges = 40
random_graph = generate_random_graph(num_nodes, num_edges)

print(f"Random graph with {num_nodes} nodes and {num_edges} edges:")
for node, neighbors in random_graph.items():
    print(f"{node}: {neighbors}")

# --- Comb forest construction (no extra front nodes, only u-v keys) ---
processed_edges = set()
CombForest = {}
last_attach = {}   # current attach point for each original node u
prev_u = None      # oldU

# initialize CombForest for original nodes
for n in random_graph.keys():
    CombForest[n] = {'parentleft': None, 'parentright': None, 'children': []}
    last_attach[n] = n

# process each undirected edge only once
for node, neighbors in random_graph.items():
    u = node
    for neighbor in neighbors:
        edge_key = f"{node}-{neighbor}" if node < neighbor else f"{neighbor}-{node}"
        if edge_key in processed_edges:
            continue
        processed_edges.add(edge_key)

        # create the edge node in the CombForest
        CombForest[edge_key] = {'parentleft': None, 'parentright': None, 'children': []}

        v = neighbor

        # ensure v exists (should), then find its leftmost attach
        if v not in CombForest:
            CombForest[v] = {'parentleft': None, 'parentright': None, 'children': []}
            last_attach[v] = v
        attach_v = find_left_end(v, CombForest)  # always ascend left for v

        # get current attach for u (the front)
        if u not in CombForest:
            CombForest[u] = {'parentleft': None, 'parentright': None, 'children': []}
            last_attach[u] = u
        current_attach_u = last_attach[u]

        # If previous u is different, "go left" on u's front by making this edge the parentleft
        print("prev_u",prev_u)
        print("u",u)
        if prev_u != u:
            print(" i got here")
            # make the new edge the left parent of current_attach_u
            current_attach_u = find_left_end(u, CombForest)
            print("current_attach_u",current_attach_u,"=",edge_key)

            CombForest[current_attach_u]['parentleft'] = edge_key
            CombForest[edge_key]['children'].append(current_attach_u)
        else:
            # prev_u is same as u (or None) -> attach the edge to the attach point
            if current_attach_u == u:
                CombForest[current_attach_u]['parentleft'] = edge_key
            else:
                CombForest[current_attach_u]['parentright'] = edge_key
            CombForest[edge_key]['children'].append(current_attach_u)

        # Attach the v-side (always use left ascend result attach_v)

        attach_v = find_left_end(v, CombForest)
        CombForest[attach_v]['parentleft'] = edge_key
        CombForest[edge_key]['children'].append(attach_v)

        # After attaching, the edge becomes the new attach point on u's front
        last_attach[u] = edge_key

        # update prev_u
        prev_u = u

def compute_max_depth(CombForest):
    # Helper to compute max ascent depth from all original nodes (measure of imbalance)
    def ascent_depth(node):
        if node not in CombForest:
            return 0
        depth_left = 1 + ascent_depth(CombForest[node]['parentleft']) if CombForest[node]['parentleft'] else 0
        depth_right = 1 + ascent_depth(CombForest[node]['parentright']) if CombForest[node]['parentright'] else 0
        return max(depth_left, depth_right)

    original_nodes = [k for k in CombForest if '-' not in k]
    return max(ascent_depth(node) for node in original_nodes)


# finished, print summary
print("\nCombForest (summary):")
for k, v in CombForest.items():
    print(f"{k}: {v}")
max_depth = compute_max_depth(CombForest)
print(f"Max ascent depth Original: {max_depth}")

Random graph with 10 nodes and 40 edges:
A: ['J', 'H', 'C', 'G', 'B', 'D', 'F', 'E']
B: ['C', 'H', 'D', 'I', 'G', 'A', 'J', 'F', 'E']
C: ['J', 'B', 'A', 'I', 'G', 'E', 'F', 'H', 'D']
D: ['B', 'E', 'H', 'A', 'I', 'G', 'F', 'C']
E: ['J', 'C', 'F', 'D', 'H', 'G', 'A', 'I', 'B']
F: ['I', 'C', 'E', 'A', 'B', 'J', 'D', 'H']
G: ['H', 'A', 'C', 'B', 'J', 'E', 'D']
H: ['G', 'A', 'B', 'J', 'E', 'D', 'I', 'C', 'F']
I: ['F', 'C', 'B', 'H', 'D', 'E']
J: ['A', 'C', 'E', 'H', 'B', 'G', 'F']
prev_u None
u A
 i got here
current_attach_u A = A-J
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u A
prev_u A
u B
 i got here
current_attach_u A-B = B-C
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u B
prev_u B
u C
 i got here
current_attach_u B-C = C-J
prev_u C
u C
prev_u C
u C
prev_u C
u C
prev_u C
u C
prev_u C
u C
prev_u C
u C
prev_u C
u D
 i got here
current_attach_u C-D = D-E
prev_u D
u D
prev_u D
u D
prev_u D
u D
prev_u D
u 

# Process Nodes in Order of Decreasing Degree During Construction(non consistent results sometimes it lowers the depth some times not )

In [ ]:


def find_left_end(curr, CombForest):
    while CombForest[curr]['parentleft'] is not None:
        curr = CombForest[curr]['parentleft']
    return curr

def find_right_end(curr, CombForest):
    while CombForest[curr]['parentright'] is not None:
        curr = CombForest[curr]['parentright']
    return curr

def build_comb_forest_degree_order(graph):
    # Step 1: Sort nodes by degree descending
    node_degrees = {node: len(neighbors) for node, neighbors in graph.items()}
    sorted_nodes = sorted(node_degrees, key=node_degrees.get, reverse=True)  # High degree first

    processed_edges = set()
    CombForest = {}
    last_attach = {}
    prev_u = None

    # Initialize CombForest for original nodes
    for n in graph.keys():
        CombForest[n] = {'parentleft': None, 'parentright': None, 'children': []}
        last_attach[n] = n

    # Process nodes in sorted order
    for node in sorted_nodes:
        u = node
        for neighbor in graph[u]:
            edge_key = f"{u}-{neighbor}" if u < neighbor else f"{neighbor}-{u}"
            if edge_key in processed_edges:
                continue
            processed_edges.add(edge_key)

            CombForest[edge_key] = {'parentleft': None, 'parentright': None, 'children': []}

            v = neighbor
            attach_v = find_left_end(v, CombForest)

            current_attach_u = last_attach[u]

            if prev_u != u:
                current_attach_u = find_left_end(u, CombForest)
                CombForest[current_attach_u]['parentleft'] = edge_key
                CombForest[edge_key]['children'].append(current_attach_u)
            else:
                if current_attach_u == u:
                    CombForest[current_attach_u]['parentleft'] = edge_key
                else:
                    CombForest[current_attach_u]['parentright'] = edge_key
                CombForest[edge_key]['children'].append(current_attach_u)

            attach_v = find_left_end(v, CombForest)
            CombForest[attach_v]['parentleft'] = edge_key
            CombForest[edge_key]['children'].append(attach_v)

            last_attach[u] = edge_key
            prev_u = u

    return CombForest

def compute_max_depth(CombForest):
    # Helper to compute max ascent depth from all original nodes (measure of imbalance)
    def ascent_depth(node):
        if node not in CombForest:
            return 0
        depth_left = 1 + ascent_depth(CombForest[node]['parentleft']) if CombForest[node]['parentleft'] else 0
        depth_right = 1 + ascent_depth(CombForest[node]['parentright']) if CombForest[node]['parentright'] else 0
        return max(depth_left, depth_right)

    original_nodes = [k for k in CombForest if '-' not in k]
    return max(ascent_depth(node) for node in original_nodes)
from collections import deque

def find_neighbors(u, CombForest):
    neighbors = set()
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            x, y = curr.split('-')
            if u == x:
                neighbors.add(y)
            elif u == y:
                neighbors.add(x)
    return neighbors

def get_all_neighbors_from_comb(CombForest):
    original_nodes = [k for k in CombForest if '-' not in k]
    neighbors_dict = {}
    for node in original_nodes:
        neighbors_dict[node] = sorted(list(find_neighbors(node, CombForest)))
    return neighbors_dict
def test_idea1(graph):

    comb_forest = build_comb_forest_degree_order(graph)
    print("CombForest Summary (Idea 1):")
    for k, v in comb_forest.items():
        print(f"{k}: {v}")
    max_depth = compute_max_depth(comb_forest)
    print(f"Max ascent depth  Decreasing Degree : {max_depth}")

# Run test
test_idea1(random_graph)

CombForest Summary (Idea 1):
A: {'parentleft': 'A-B', 'parentright': None, 'children': []}
B: {'parentleft': 'B-C', 'parentright': None, 'children': []}
C: {'parentleft': 'B-C', 'parentright': None, 'children': []}
D: {'parentleft': 'B-D', 'parentright': None, 'children': []}
E: {'parentleft': 'B-E', 'parentright': None, 'children': []}
F: {'parentleft': 'B-F', 'parentright': None, 'children': []}
G: {'parentleft': 'B-G', 'parentright': None, 'children': []}
H: {'parentleft': 'B-H', 'parentright': None, 'children': []}
I: {'parentleft': 'B-I', 'parentright': None, 'children': []}
J: {'parentleft': 'B-J', 'parentright': None, 'children': []}
B-C: {'parentleft': 'C-J', 'parentright': 'B-H', 'children': ['B', 'C']}
B-H: {'parentleft': 'C-H', 'parentright': 'B-D', 'children': ['B-C', 'H']}
B-D: {'parentleft': 'C-D', 'parentright': 'B-I', 'children': ['B-H', 'D']}
B-I: {'parentleft': 'C-I', 'parentright': 'B-G', 'children': ['B-D', 'I']}
B-G: {'parentleft': 'C-G', 'parentright': 'A-B', 'chi

# Build Balanced Binary Trees for Each Node's Incident Edges(not working)

In [ ]:
import random

# ---------------------------
# Utility: safe left-end finder
# ---------------------------
def find_left_end(curr, CombForest):
    """
    Walk parentleft pointers until None.
    Cycle-protected: if we revisit a node, break.
    """
    seen = set()
    while CombForest[curr]['parentleft'] is not None:
        if curr in seen:  # cycle detected
            break
        seen.add(curr)
        curr = CombForest[curr]['parentleft']
    return curr

# ---------------------------
# Balanced comb builder
# ---------------------------
def build_balanced_comb_for_u(u, neighbors, CombForest, last_attach):
    """
    For one node u, build a balanced BST of edge nodes 'u-v' over its incident neighbors.
    """
    sorted_neighbors = sorted(neighbors)

    if not sorted_neighbors:
        return

    def ensure_node(n):
        if n not in CombForest:
            CombForest[n] = {'parentleft': None, 'parentright': None, 'children': []}

    def add_child(parent_key, child_key):
        if child_key not in CombForest[parent_key]['children']:
            CombForest[parent_key]['children'].append(child_key)

    def set_parentleft(child_key, parent_key):
        if CombForest[child_key]['parentleft'] is None and child_key != parent_key:
            CombForest[child_key]['parentleft'] = parent_key
            add_child(parent_key, child_key)

    def set_parentright(child_key, parent_key):
        if CombForest[child_key]['parentright'] is None and child_key != parent_key:
            CombForest[child_key]['parentright'] = parent_key
            add_child(parent_key, child_key)

    # Recursive balanced BST builder
    def build_bst(low, high):
        if low > high:
            return None

        mid = (low + high) // 2
        v = sorted_neighbors[mid]

        edge_key = f"{u}-{v}" if u < v else f"{v}-{u}"
        ensure_node(edge_key)
        ensure_node(v)

        # Attach v’s chain under this edge
        attach_v = find_left_end(v, CombForest)
        if (CombForest[attach_v]['parentleft'] is None and
            CombForest[attach_v]['parentright'] is None):
            CombForest[attach_v]['parentleft'] = edge_key
            add_child(edge_key, attach_v)

        # Recurse left/right
        left_root = build_bst(low, mid - 1)
        right_root = build_bst(mid + 1, high)

        if left_root:
            set_parentleft(left_root, edge_key)
        if right_root:
            set_parentright(right_root, edge_key)

        return edge_key

    # Build and hang above u
    ensure_node(u)
    root_edge = build_bst(0, len(sorted_neighbors) - 1)

    if root_edge is not None:
        current_attach_u = last_attach[u]
        if CombForest[current_attach_u]['parentleft'] is None:
            CombForest[current_attach_u]['parentleft'] = root_edge
            add_child(root_edge, current_attach_u)
        last_attach[u] = root_edge

# ---------------------------
# Forest builder
# ---------------------------
def build_comb_forest_balanced_bst(graph):
    CombForest = {}
    last_attach = {}
    processed_edges = set()

    # init nodes
    for n in graph.keys():
        CombForest[n] = {'parentleft': None, 'parentright': None, 'children': []}
        last_attach[n] = n

    # process each node
    for u in sorted(graph.keys()):
        unprocessed_neighbors = []
        for v in graph[u]:
            ek = f"{min(u,v)}-{max(u,v)}"
            if ek not in processed_edges:
                processed_edges.add(ek)
                unprocessed_neighbors.append(v)

        if unprocessed_neighbors:
            build_balanced_comb_for_u(u, unprocessed_neighbors, CombForest, last_attach)

    return CombForest

# ---------------------------
# Depth calculator
# ---------------------------
def compute_max_depth(CombForest):
    roots = [n for n, d in CombForest.items() if d['parentleft'] is None and d['parentright'] is None]
    if not roots:
        roots = list(CombForest.keys())

    def dfs(node, seen):
        if node in seen:
            return 0
        seen.add(node)
        if not CombForest[node]['children']:
            seen.remove(node)
            return 1
        best = 1
        for ch in CombForest[node]['children']:
            best = max(best, 1 + dfs(ch, seen))
        seen.remove(node)
        return best

    return max(dfs(r, set()) for r in roots)

# ---------------------------
# Example usage
# ---------------------------
# Small test graph
random_graph = {
    '1': ['2','3','4'],
    '2': ['1','4','6'],
    '3': ['1'],
    '4': ['1','2'],
    '6': ['2']
}

comb_forest = build_comb_forest_balanced_bst(random_graph)

print("CombForest Summary (Idea 2):")
for k,v in comb_forest.items():
    print(f"{k}: {v}")
print("Max ascent depth:", compute_max_depth(comb_forest))


CombForest Summary (Idea 2):
1: {'parentleft': '1-3', 'parentright': None, 'children': []}
2: {'parentleft': '1-2', 'parentright': None, 'children': []}
3: {'parentleft': '1-3', 'parentright': None, 'children': []}
4: {'parentleft': '1-4', 'parentright': None, 'children': []}
6: {'parentleft': '2-6', 'parentright': None, 'children': []}
1-3: {'parentleft': None, 'parentright': None, 'children': ['3', '1-2', '1-4', '1']}
1-2: {'parentleft': '1-3', 'parentright': None, 'children': ['2']}
1-4: {'parentleft': None, 'parentright': '1-3', 'children': ['4']}
2-4: {'parentleft': None, 'parentright': None, 'children': ['2-6']}
2-6: {'parentleft': None, 'parentright': '2-4', 'children': ['6']}
Max ascent depth: 3


# Post-Construction Rotations Like in AVL Trees(to be adjusted to weight)

In [ ]:
import random

# Reuse generate_random_graph, find_left_end, find_right_end, and original build_comb_forest (rename for clarity)

def build_comb_forest_original(graph):
    # The original construction code from the query, pasted here for completeness
    processed_edges = set()
    CombForest = {}
    last_attach = {}
    prev_u = None

    for n in graph.keys():
        CombForest[n] = {'parentleft': None, 'parentright': None, 'children': []}
        last_attach[n] = n

    for node in graph.keys():
        u = node
        for neighbor in graph[u]:
            edge_key = f"{node}-{neighbor}" if node < neighbor else f"{neighbor}-{node}"
            if edge_key in processed_edges:
                continue
            processed_edges.add(edge_key)

            CombForest[edge_key] = {'parentleft': None, 'parentright': None, 'children': []}

            v = neighbor
            attach_v = find_left_end(v, CombForest)

            current_attach_u = last_attach[u]

            if prev_u != u:
                current_attach_u = find_left_end(u, CombForest)
                CombForest[current_attach_u]['parentleft'] = edge_key
                CombForest[edge_key]['children'].append(current_attach_u)
            else:
                if current_attach_u == u:
                    CombForest[current_attach_u]['parentleft'] = edge_key
                else:
                    CombForest[current_attach_u]['parentright'] = edge_key
                CombForest[edge_key]['children'].append(current_attach_u)

            attach_v = find_left_end(v, CombForest)
            CombForest[attach_v]['parentleft'] = edge_key
            CombForest[edge_key]['children'].append(attach_v)

            last_attach[u] = edge_key
            prev_u = u

    return CombForest

def get_height(node, CombForest, height_memo):
    if node is None:
        return 0
    if node in height_memo:
        return height_memo[node]
    # Height is 1 + max of children's heights (descent view)
    children = CombForest[node]['children']
    height = 1 + max(get_height(c, CombForest, height_memo) for c in children) if children else 1
    height_memo[node] = height
    return height

def rotate_right(y, CombForest):
    x = CombForest[y]['parentleft']  # Assuming parentleft is 'left child' in descent view
    if x is None:
        return y
    # Standard AVL right rotation (adjust for our pointers)
    CombForest[y]['parentleft'] = CombForest[x]['parentright']
    if CombForest[x]['parentright']:
        CombForest[CombForest[x]['parentright']]['parent'] = y  # Need to track parents? Add 'parent' field if needed
    # Note: To make rotations work, we may need to add a 'parent' field or invert view. For simplicity, assume we update children accordingly.
    CombForest[x]['parentright'] = y
    # Update children lists (swap)
    CombForest[y]['children'] = [c for c in CombForest[y]['children'] if c != x] + (CombForest[x]['children'] if 'children' in CombForest[x] else [])
    CombForest[x]['children'].append(y)
    return x

# Similar for left_rotate

def balance_comb_forest(CombForest):
    height_memo = {}
    # Find all nodes and balance bottom-up (post-order traversal)
    def balance_node(node):
        if node is None:
            return None
        # Balance children first
        CombForest[node]['parentleft'] = balance_node(CombForest[node]['parentleft'])
        CombForest[node]['parentright'] = balance_node(CombForest[node]['parentright'])

        # Compute balance factor (left - right height)
        left_h = get_height(CombForest[node]['parentleft'], CombForest, height_memo)
        right_h = get_height(CombForest[node]['parentright'], CombForest, height_memo)
        balance = left_h - right_h

        # Rotations
        if balance > 1:
            # Left heavy
            if get_height(CombForest[CombForest[node]['parentleft']]['parentleft'], CombForest, height_memo) < get_height(CombForest[CombForest[node]['parentleft']]['parentright'], CombForest, height_memo):
                CombForest[node]['parentleft'] = left_rotate(CombForest[node]['parentleft'], CombForest)  # Left-right case
            return right_rotate(node, CombForest)
        if balance < -1:
            # Right heavy
            if get_height(CombForest[CombForest[node]['parentright']]['parentright'], CombForest, height_memo) < get_height(CombForest[CombForest[node]['parentright']]['parentleft'], CombForest, height_memo):
                CombForest[node]['parentright'] = right_rotate(CombForest[node]['parentright'], CombForest)  # Right-left case
            return left_rotate(node, CombForest)
        return node

    # Start from all edge nodes (roots have no parents)
    roots = [k for k in CombForest if CombForest[k]['parentleft'] is None and CombForest[k]['parentright'] is None and '-' in k]
    for root in roots:
        balance_node(root)
    return CombForest

def left_rotate(x, CombForest):
    y = CombForest[x]['parentright']
    if y is None:
        return x
    CombForest[x]['parentright'] = CombForest[y]['parentleft']
    if CombForest[y]['parentleft']:
        # Update parent of the subtree
        pass  # Assume update
    CombForest[y]['parentleft'] = x
    # Update children
    CombForest[x]['children'] = [c for c in CombForest[x]['children'] if c != y] + (CombForest[y]['children'] if 'children' in CombForest[y] else [])
    CombForest[y]['children'].append(x)
    return y



def test_idea3(graph):

    comb_forest = build_comb_forest_original(graph)
    comb_forest = balance_comb_forest(comb_forest)
    print("CombForest Summary (Idea 3, after balancing):")
    for k, v in comb_forest.items():
        print(f"{k}: {v}")
    max_depth = compute_max_depth(comb_forest)
    print(f"Max ascent depth Post-Construction Rotations: {max_depth}")


test_idea3(random_graph)

CombForest Summary (Idea 3, after balancing):
A: {'parentleft': 'A-J', 'parentright': None, 'children': []}
B: {'parentleft': 'A-B', 'parentright': None, 'children': []}
C: {'parentleft': 'A-C', 'parentright': None, 'children': []}
D: {'parentleft': 'A-D', 'parentright': None, 'children': []}
E: {'parentleft': 'A-E', 'parentright': None, 'children': []}
F: {'parentleft': 'A-F', 'parentright': None, 'children': []}
G: {'parentleft': 'A-G', 'parentright': None, 'children': []}
H: {'parentleft': 'A-H', 'parentright': None, 'children': []}
I: {'parentleft': 'B-I', 'parentright': None, 'children': []}
J: {'parentleft': 'A-J', 'parentright': None, 'children': []}
A-J: {'parentleft': 'B-J', 'parentright': 'A-H', 'children': ['A', 'J']}
A-H: {'parentleft': 'B-H', 'parentright': 'A-C', 'children': ['A-J', 'H']}
A-C: {'parentleft': 'B-C', 'parentright': 'A-G', 'children': ['A-H', 'C']}
A-G: {'parentleft': 'B-G', 'parentright': 'A-B', 'children': ['A-C', 'G']}
A-B: {'parentleft': 'B-C', 'parentri

# Height-Based Attachment(failed)

In [ ]:
import random

# Reuse generate_random_graph, find_left_end, find_right_end

def update_height(node, CombForest, visited=None):
    if node is None:
        return 0
    if visited is None:
        visited = set()
    if node in visited:
        return CombForest[node].get('height', 1)  # avoid infinite loop
    visited.add(node)

    left = CombForest[node]['parentleft']
    right = CombForest[node]['parentright']

    left_h = update_height(left, CombForest, visited) if left else 0
    right_h = update_height(right, CombForest, visited) if right else 0

    CombForest[node]['height'] = 1 + max(left_h, right_h)
    return CombForest[node]['height']

def build_comb_forest_dynamic_balance(graph):
    processed_edges = set()
    CombForest = {}
    last_attach = {}
    prev_u = None

    for n in graph.keys():
        CombForest[n] = {'parentleft': None, 'parentright': None, 'children': [], 'height': 1}
        last_attach[n] = n

    for node in graph.keys():
        u = node
        for neighbor in graph[u]:
            edge_key = f"{node}-{neighbor}" if node < neighbor else f"{neighbor}-{node}"
            if edge_key in processed_edges:
                continue
            processed_edges.add(edge_key)

            CombForest[edge_key] = {'parentleft': None, 'parentright': None, 'children': [], 'height': 1}

            v = neighbor
            attach_v = find_left_end(v, CombForest)

            current_attach_u = last_attach[u]

            if prev_u != u:
                current_attach_u = find_left_end(u, CombForest)
                CombForest[current_attach_u]['parentleft'] = edge_key
                CombForest[edge_key]['children'].append(current_attach_u)
            else:
                # Dynamic choice: Attach to the shorter side
                left_h = CombForest[current_attach_u].get('height_left', 0)  # Need to track sub-heights
                right_h = CombForest[current_attach_u].get('height_right', 0)
                if left_h <= right_h:
                    CombForest[current_attach_u]['parentleft'] = edge_key
                    CombForest[current_attach_u]['height_left'] = 1  # Update sub-heights
                else:
                    CombForest[current_attach_u]['parentright'] = edge_key
                    CombForest[current_attach_u]['height_right'] = 1
                CombForest[edge_key]['children'].append(current_attach_u)

            attach_v = find_left_end(v, CombForest)
            CombForest[attach_v]['parentleft'] = edge_key
            CombForest[edge_key]['children'].append(attach_v)

            # Update heights after attachment
            update_height(edge_key, CombForest)

            last_attach[u] = edge_key
            prev_u = u

    return CombForest

# Modify compute_max_depth to use 'height' field
def compute_max_depth(CombForest):
    return max(data['height'] for data in CombForest.values())

def test_idea4(graph):

    comb_forest = build_comb_forest_dynamic_balance(graph)
    print("CombForest Summary (Idea 4):")
    for k, v in comb_forest.items():
        print(f"{k}: {v}")
    max_depth = compute_max_depth(comb_forest)
    print(f"Max height: {max_depth}")

test_idea4(random_graph)

KeyboardInterrupt: 

In [ ]:

from collections import defaultdict, deque
import random

class ActivationNeighborFinder:
    def __init__(self, CombForest):
        self.CombForest = CombForest

    # Method 1: Threshold-based Activation with Decay
    def find_neighbors_threshold_activation(self, u, threshold=0.5, decay_factor=0.7):
        """
        Nodes activate based on energy threshold. Energy decays as it spreads.
        Edge nodes that reach threshold reveal their endpoints as neighbors.
        """
        activation = defaultdict(float)
        neighbors = set()

        # Initial activation
        activation[u] = 1.0
        queue = deque([(u, 1.0)])
        visited = set()

        while queue:
            curr, energy = queue.popleft()
            if curr in visited or energy < threshold:
                continue
            visited.add(curr)

            # If this is an edge node and has enough energy, extract neighbors
            if '-' in curr and energy >= threshold:
                x, y = curr.split('-')
                if u == x:
                    neighbors.add(y)
                elif u == y:
                    neighbors.add(x)

            # Propagate energy to connected nodes
            new_energy = energy * decay_factor
            for parent_key in ['parentleft', 'parentright']:
                parent = self.CombForest[curr].get(parent_key)
                if parent and parent not in visited:
                    activation[parent] = max(activation[parent], new_energy)
                    queue.append((parent, new_energy))

        return neighbors, activation

    # Method 2: Pulse-based Activation with Time Steps
    def find_neighbors_pulse_activation(self, u, max_steps=5):
        """
        Send pulses through the network. Each time step, activated nodes
        send signals to their connections. Edge nodes "fire" when activated.
        """
        current_active = {u}
        neighbors = set()
        all_activations = []  # Track activation history

        for step in range(max_steps):
            next_active = set()
            step_activation = {}

            for node in current_active:
                step_activation[node] = 1.0

                # If edge node is active, extract neighbor
                if '-' in node:
                    x, y = node.split('-')
                    if u == x:
                        neighbors.add(y)
                    elif u == y:
                        neighbors.add(x)

                # Propagate to connected nodes
                for parent_key in ['parentleft', 'parentright']:
                    parent = self.CombForest[node].get(parent_key)
                    if parent:
                        next_active.add(parent)
                        step_activation[parent] = step_activation.get(parent, 0) + 0.8

            all_activations.append(step_activation)
            current_active = next_active

            if not current_active:
                break

        return neighbors, all_activations

    # Method 3: Competitive Activation (Winner-Take-All)
    def find_neighbors_competitive_activation(self, u, competition_rounds=3):
        """
        Multiple "signals" compete to traverse the network. Only strongest
        signals survive each round. Winning signals that reach edge nodes
        reveal neighbors.
        """
        # Initialize competing signals with different strengths
        signals = {
            f"signal_{i}": {
                'position': u,
                'strength': random.uniform(0.5, 1.0),
                'path': [u]
            } for i in range(5)
        }

        neighbors = set()

        for round_num in range(competition_rounds):
            new_signals = {}

            for sig_id, signal in signals.items():
                curr_pos = signal['position']

                # Check if current position is an edge node
                if '-' in curr_pos:
                    x, y = curr_pos.split('-')
                    if u == x:
                        neighbors.add(y)
                    elif u == y:
                        neighbors.add(x)

                # Move signal to connected nodes
                possible_moves = []
                for parent_key in ['parentleft', 'parentright']:
                    parent = self.CombForest[curr_pos].get(parent_key)
                    if parent and parent not in signal['path']:  # Avoid cycles
                        possible_moves.append(parent)

                # Create new signals for each possible move
                for move in possible_moves:
                    new_strength = signal['strength'] * 0.85  # Decay
                    new_sig_id = f"{sig_id}_{move}"
                    new_signals[new_sig_id] = {
                        'position': move,
                        'strength': new_strength,
                        'path': signal['path'] + [move]
                    }

            # Competition: keep only strongest signals (max 10)
            if len(new_signals) > 10:
                sorted_signals = sorted(new_signals.items(),
                                      key=lambda x: x[1]['strength'],
                                      reverse=True)
                signals = dict(sorted_signals[:10])
            else:
                signals = new_signals

        return neighbors, signals

    # Method 4: Resonance-based Activation
    def find_neighbors_resonance_activation(self, u, resonance_freq=0.6):
        """
        Nodes have different resonance frequencies. Only nodes that resonate
        with the query frequency get activated and can reveal information.
        """
        # Assign resonance frequencies to nodes
        node_frequencies = {}
        for node in self.CombForest.keys():
            if '-' in node:
                # Edge nodes have frequencies based on their endpoints
                x, y = node.split('-')
                node_frequencies[node] = (hash(x) + hash(y)) % 100 / 100.0
            else:
                node_frequencies[node] = hash(node) % 100 / 100.0

        activation_strength = {}
        neighbors = set()
        queue = deque([u])
        visited = set()

        while queue:
            curr = queue.popleft()
            if curr in visited:
                continue
            visited.add(curr)

            # Calculate resonance activation
            node_freq = node_frequencies.get(curr, 0.5)
            resonance = 1.0 - abs(resonance_freq - node_freq)
            activation_strength[curr] = resonance

            # Only process if resonance is strong enough
            if resonance > 0.7:
                if '-' in curr:
                    x, y = curr.split('-')
                    if u == x:
                        neighbors.add(y)
                    elif u == y:
                        neighbors.add(x)

                # Propagate to connected nodes
                for parent_key in ['parentleft', 'parentright']:
                    parent = self.CombForest[curr].get(parent_key)
                    if parent and parent not in visited:
                        queue.append(parent)

        return neighbors, activation_strength

    # Method 5: Multi-layer Activation with Inhibition
    def find_neighbors_inhibition_activation(self, u):
        """
        Nodes can be activated or inhibited. Inhibited nodes block signal
        propagation. Creates more selective neighbor discovery.
        """
        activation_state = defaultdict(lambda: 'neutral')  # 'active', 'inhibited', 'neutral'
        neighbors = set()

        # Start with target node active
        activation_state[u] = 'active'

        # Randomly inhibit some nodes to create selective paths
        all_nodes = list(self.CombForest.keys())
        inhibited_nodes = random.sample(all_nodes, min(3, len(all_nodes) // 4))
        for node in inhibited_nodes:
            if node != u:
                activation_state[node] = 'inhibited'

        queue = deque([u])
        visited = set()

        while queue:
            curr = queue.popleft()
            if curr in visited or activation_state[curr] == 'inhibited':
                continue
            visited.add(curr)

            # Activate current node
            activation_state[curr] = 'active'

            if '-' in curr:
                x, y = curr.split('-')
                if u == x:
                    neighbors.add(y)
                elif u == y:
                    neighbors.add(x)

            # Propagate only if not inhibited
            for parent_key in ['parentleft', 'parentright']:
                parent = self.CombForest[curr].get(parent_key)
                if (parent and parent not in visited and
                    activation_state[parent] != 'inhibited'):
                    queue.append(parent)

        return neighbors, dict(activation_state)

    # Method 6: Mutual Neighbors with Cross-Activation
    def find_mutual_neighbors_cross_activation(self, u, v, sync_threshold=0.8):
        """
        Find mutual neighbors by having two activation processes (from u and v)
        that must synchronize at edge nodes to reveal mutual connections.
        """
        # Run dual activation processes
        activation_u = defaultdict(float)
        activation_v = defaultdict(float)
        mutual_neighbors = set()

        # Initialize
        activation_u[u] = 1.0
        activation_v[v] = 1.0

        # Simultaneous BFS from both nodes
        queue_u = deque([(u, 1.0)])
        queue_v = deque([(v, 1.0)])
        visited_u = set()
        visited_v = set()

        max_steps = 10
        step = 0

        while (queue_u or queue_v) and step < max_steps:
            step += 1

            # Process u's activation
            if queue_u:
                curr_u, energy_u = queue_u.popleft()
                if curr_u not in visited_u and energy_u > 0.1:
                    visited_u.add(curr_u)
                    activation_u[curr_u] = energy_u

                    for parent_key in ['parentleft', 'parentright']:
                        parent = self.CombForest[curr_u].get(parent_key)
                        if parent and parent not in visited_u:
                            queue_u.append((parent, energy_u * 0.8))

            # Process v's activation
            if queue_v:
                curr_v, energy_v = queue_v.popleft()
                if curr_v not in visited_v and energy_v > 0.1:
                    visited_v.add(curr_v)
                    activation_v[curr_v] = energy_v

                    for parent_key in ['parentleft', 'parentright']:
                        parent = self.CombForest[curr_v].get(parent_key)
                        if parent and parent not in visited_v:
                            queue_v.append((parent, energy_v * 0.8))

        # Find nodes where both activations are strong (synchronized)
        for node in set(activation_u.keys()) & set(activation_v.keys()):
            combined_strength = activation_u[node] + activation_v[node]
            if combined_strength > sync_threshold and '-' in node:
                x, y = node.split('-')
                # This edge connects to both u and v, so x and y are mutual neighbors
                if u != x and v != x:
                    mutual_neighbors.add(x)
                if u != y and v != y:
                    mutual_neighbors.add(y)

        return mutual_neighbors, (activation_u, activation_v)

# Example usage functions
def demonstrate_activation_methods(CombForest, test_node):
    """Demonstrate all activation methods"""
    finder = ActivationNeighborFinder(CombForest)

    print(f"\n=== Activation Methods for Node '{test_node}' ===")

    # Method 1: Threshold activation
    neighbors1, activation1 = finder.find_neighbors_threshold_activation(test_node)
    print(f"1. Threshold Activation: {neighbors1}")

    # Method 2: Pulse activation
    neighbors2, pulses = finder.find_neighbors_pulse_activation(test_node)
    print(f"2. Pulse Activation: {neighbors2}")

    # Method 3: Competitive activation
    neighbors3, signals = finder.find_neighbors_competitive_activation(test_node)
    print(f"3. Competitive Activation: {neighbors3}")

    # Method 4: Resonance activation
    neighbors4, resonance = finder.find_neighbors_resonance_activation(test_node)
    print(f"4. Resonance Activation: {neighbors4}")

    # Method 5: Inhibition activation
    neighbors5, states = finder.find_neighbors_inhibition_activation(test_node)
    print(f"5. Inhibition Activation: {neighbors5}")

    # Compare with original method
    original_neighbors = find_neighbors_original(test_node, CombForest)
    print(f"Original Method: {original_neighbors}")

    return {
        'threshold': neighbors1,
        'pulse': neighbors2,
        'competitive': neighbors3,
        'resonance': neighbors4,
        'inhibition': neighbors5,
        'original': original_neighbors
    }

def find_neighbors_original(u, CombForest):
    """Original neighbor finding method for comparison"""
    neighbors = set()
    from collections import deque
    queue = deque([u])
    visited = set()
    while queue:
        curr = queue.popleft()
        if curr in visited:
            continue
        visited.add(curr)
        pl = CombForest[curr]['parentleft']
        if pl is not None and pl not in visited:
            queue.append(pl)
        pr = CombForest[curr]['parentright']
        if pr is not None and pr not in visited:
            queue.append(pr)
        if '-' in curr:
            x, y = curr.split('-')
            if u == x:
                neighbors.add(y)
            elif u == y:
                neighbors.add(x)
    return neighbors
Results=demonstrate_activation_methods(CombForest, 'A')
print(Results)
Resul=demonstrate_activation_methods(CombForest, 'B')
print(Resul)
Mn=find_mutual_neighbors_cross_activation(CombForest, 'A', 'B', sync_threshold=0.8)
Print (Mn)


=== Activation Methods for Node 'A' ===
1. Threshold Activation: {'J'}
2. Pulse Activation: {'H', 'J', 'G', 'C'}
3. Competitive Activation: {'H', 'J'}
4. Resonance Activation: {'J'}
5. Inhibition Activation: {'H', 'J', 'C'}
Original Method: {'E', 'B', 'H', 'J', 'D', 'G', 'C', 'F'}
{'threshold': {'J'}, 'pulse': {'H', 'J', 'G', 'C'}, 'competitive': {'H', 'J'}, 'resonance': {'J'}, 'inhibition': {'H', 'J', 'C'}, 'original': {'E', 'B', 'H', 'J', 'D', 'G', 'C', 'F'}}

=== Activation Methods for Node 'B' ===
1. Threshold Activation: {'A'}
2. Pulse Activation: {'H', 'D', 'C', 'A', 'F', 'I'}
3. Competitive Activation: {'C', 'A'}
4. Resonance Activation: set()
5. Inhibition Activation: {'E', 'H', 'J', 'D', 'G', 'C', 'A', 'F', 'I'}
Original Method: {'E', 'H', 'J', 'D', 'G', 'C', 'A', 'F', 'I'}
{'threshold': {'A'}, 'pulse': {'H', 'D', 'C', 'A', 'F', 'I'}, 'competitive': {'C', 'A'}, 'resonance': set(), 'inhibition': {'E', 'H', 'J', 'D', 'G', 'C', 'A', 'F', 'I'}, 'original': {'E', 'H', 'J', 'D', 'G

NameError: name 'find_mutual_neighbors_cross_activation' is not defined